In [1]:
from copy import copy
from random import choice
from scipy.stats import poisson, skellam
from itertools import product, chain

In [2]:
# Constantes
RENT_REWARD = 10
MOVE_CAR_COST = 2
MAX_MOVE_CARS = 5
TOTAL_CARS_BY_LOCATION = 20

# λ para a distribuição de probabilidade Poisson do número
# de retornos e pedidos de aluguel
FIRST_LOCATION_REQUEST_POISSON_LAMBDA = 3
FIRST_LOCATION_RETURN_POISSON_LAMBDA = 3
SECOND_LOCATION_REQUEST_POISSON_LAMBDA = 4
SECOND_LOCATION_RETURN_POISSON_LAMBDA = 2

# Hyperparâmetros 
DISCOUNT_FACTOR = 0.9
ERROR_THRESHOLD = 0.001
MAX_POLICY_IMPROVEMENT_ITERATIONS = 10 ** 5

FIRST_LOCATION_RENTED_CARS_CALC_RANGE = tuple(int(n) for n in poisson.interval(0.999, FIRST_LOCATION_REQUEST_POISSON_LAMBDA))
SECOND_LOCATION_RENTED_CARS_CALC_RANGE = map(int, poisson.interval(0.999, SECOND_LOCATION_REQUEST_POISSON_LAMBDA))
FIRST_LOCATION_NEXT_BALANCE_CALC_RANGE = map(int, skellam.interval(0.999, FIRST_LOCATION_RETURN_POISSON_LAMBDA, 
                                                                   FIRST_LOCATION_REQUEST_POISSON_LAMBDA))
SECOND_LOCATION_NEXT_BALANCE_CALC_RANGE = map(int, skellam.interval(0.999, SECOND_LOCATION_RETURN_POISSON_LAMBDA, 
                                                                    SECOND_LOCATION_REQUEST_POISSON_LAMBDA))

In [22]:
FIRST_LOCATION_NEXT_BALANCE_CALC_RANGE = tuple(range(FIRST_LOCATION_NEXT_BALANCE_CALC_RANGE[0], 
                                                     FIRST_LOCATION_NEXT_BALANCE_CALC_RANGE[1]))
SECOND_LOCATION_NEXT_BALANCE_CALC_RANGE = tuple(range(SECOND_LOCATION_NEXT_BALANCE_CALC_RANGE[0], 
                                                      SECOND_LOCATION_NEXT_BALANCE_CALC_RANGE[1]))

TypeError: 'map' object is not subscriptable

In [3]:
# Estados
STATES_BY_LOCATION = tuple(range(TOTAL_CARS_BY_LOCATION + 1))
STATES = list(product(STATES_BY_LOCATION, STATES_BY_LOCATION))

In [4]:
def actions_set(state):
    """
    Ações possíveis dado um estado. Exemplo: Dado estado (15, 18) -> [-5, -4, -3, -2, -1, 0, 1, 2]
    """
    first_location, second_location = state
    # O número máximo de carros que pode sair de uma locadora é, o menor número entre:
    #    - O número de carros na locadora (first_location)
    #    - O número máximo de carros que a segunda locadora pode receber (TOTAL_CARS_BY_LOCATION - second_location)
    #    - O número máximo de movimentos (MAX_MOVE_CARS)
    from_first_location = min(first_location, TOTAL_CARS_BY_LOCATION - second_location, MAX_MOVE_CARS)
    from_first_location = range(1, from_first_location + 1)
    from_second_location = min(second_location, TOTAL_CARS_BY_LOCATION - first_location, MAX_MOVE_CARS)
    from_second_location = range(-1, from_second_location * -1 - 1, -1)
    return tuple(chain(from_second_location, (0,), from_first_location))

In [5]:
def rental_cars_probability_given_balance(rental_cars, balance, request_lambda, return_lambda):
    """
    Probabilidade de pedidos (rental_cars) dado o saldo final de carros (balance).
    Segundo o teorema de Bayes (Probabilidade condicional), temos:
    P(A | B) = P(B | A) * P (A) / P(B)
    P(A) -> Probabilidade da quantidade de alugueis no dia
    P(B) -> Probabilidade do saldo final de carros no dia
    P(B | A) -> Probabilidade do saldo final de carros dado a quantidade de alugueis no dia
    
    Dado a quantidade de alugueis no dia, a probabilidade do saldo final de carros é igual a probabilidade
    de carros devolvidos = saldos + quantidade de carros alugados no dia (balance + rental_cars).
    
    A probabilidade do saldo final de carros é dado pela distribuição Skellam. Skellam é a 
    dsitribuição de probabilidade da diferença entre duas variáveis aleatórias de Poission
    """
    return (poisson.pmf(balance + rental_cars, return_lambda) * poisson.pmf(rental_cars, request_lambda) / 
            skellam.pmf(balance, return_lambda, request_lambda))

In [19]:
def reward_expectation(state, action, state_values):
    """
    Recompensa total esperada dado um estado e uma ação
    """
    reward = 0
    first_location, second_location = state
    
    # Primeiro é necessário fazer a transferência de carros entre as locadoras
    first_location -= action
    second_location += action
    
    
    for balance in range(FIRST_LOCATION_NEXT_BALANCE_CALC_RANGE)

    for next_state in STATES:
        next_first_location, next_second_location = next_state
        
        # Recompensa esperada para a primeira locadora
        for rented in range(MAX_FIRST_LOCATION_RENTED_CARS_CALC):
            reward += rented * RENT_REWARD * rental_cars_probability_given_balance(rented, next_first_location - first_location,
                                                                                   FIRST_LOCATION_REQUEST_POISSON_LAMBDA,FIRST_LOCATION_RETURN_POISSON_LAMBDA)
        # Recompensa esperada para a segunda locadora
        for rented in range(MAX_SECOND_LOCATION_RENTED_CARS_CALC):
            reward += rented * RENT_REWARD * rental_cars_probability_given_balance(rented, next_first_location - first_location,
                                                                                   SECOND_LOCATION_REQUEST_POISSON_LAMBDA,
                                                                                   SECOND_LOCATION_RETURN_POISSON_LAMBDA)
            
        # Recompensa futura
        reward += DISCOUNT_FACTOR * state_values[next_state]

    return reward

In [7]:
def policy_evaluation(state_values, policy):
    """
    Avaliação da política: atualizando os valor reais de cada estado dado uma política.
    """
    error = None
    while error is None or error > ERROR_THRESHOLD:
        error = 0
        for s in STATES:
            print(s)
            value = state_values[s]
            state_values[s] = reward_expectation(s, policy[s], state_values)
            error = max(error, abs(value - state_values[s]))
            print('finish')

In [8]:
def policy_improvement(state_values, policy):
    """
    Melhora a política utilizando a função state-value como base. Escolhe as ações que vão dá as melhores recompensas.
    :return True se a política foi alterada. Caso contrário, False, significando que política já está estável. 
    """
    is_policy_stable = True
    for s in STATES:
        old_action = policy[s]
        policy[s] = max(actions_set(s), lambda a: reward_expectation(s, a, state_values))
        is_policy_stable = False if old_action != policy[s] else is_policy_stable
    return not is_policy_stable

In [20]:
# Inicialização do vetor state-value e policy
state_values = {s: 0 for s in STATES}
policy = {s: choice(actions_set(s)) for s in STATES}

is_policy_stable = False
for i in range(MAX_POLICY_IMPROVEMENT_ITERATIONS):
    print(f'Iteration {i + 1}: Policy evaluation...')
    policy_evaluation(state_values, policy)
    print(f'Iteration {i + 1}: Policy improvement...')
    if not policy_improvement(state_values, policy):
        break

Iteration 1: Policy evaluation...
(0, 0)
finish
(0, 1)
finish
(0, 2)
finish
(0, 3)
finish
(0, 4)
finish
(0, 5)
finish
(0, 6)
finish
(0, 7)
finish
(0, 8)
finish
(0, 9)
finish
(0, 10)
finish
(0, 11)
finish
(0, 12)


KeyboardInterrupt: 

In [18]:
rental_cars_probability_given_balance(5, 3,
                                      FIRST_LOCATION_REQUEST_POISSON_LAMBDA,
                                      FIRST_LOCATION_RETURN_POISSON_LAMBDA)

0.010928975257349904